In [3]:
# !pip3 install grad-cam

In [4]:
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
from torchvision import transforms
import numpy as np

In [5]:
# mango_pth = "mango_leaf_dataset/"
# mango_dir = os.listdir(mango_pth)
# print(len(mango_dir))
# print(mango_dir)
# diseased = []
# healthy = []
# for val in range(1, 6):
#     print(val)
#     diseased.append(
#         "https://raw.githubusercontent.com/Nakiyah24/ExplainableAI_Assignments/main/Assignment5/mango_leaf_dataset/disease_000%s"
#         % val
#     )
#     healthy.append(
#         "https://raw.githubusercontent.com/Nakiyah24/ExplainableAI_Assignments/main/Assignment5/mango_leaf_dataset/healthy_000%s"
#         % val
#     )

In [6]:
raw_github_path = f"https://raw.githubusercontent.com/Nakiyah24/ExplainableAI_Assignments/main/Assignment5/mango_leaf_dataset"

diseased_lst, healthy_lst = [], []
for i in range(1, 4):  # 0001..0005
    diseased_lst.append(f"{raw_github_path}/disease_{i:04d}.JPG")
    healthy_lst.append(f"{raw_github_path}/healthy_{i:04d}.JPG")

print(diseased_lst)
print(healthy_lst)

['https://raw.githubusercontent.com/Nakiyah24/ExplainableAI_Assignments/main/Assignment5/mango_leaf_dataset/disease_0001.JPG', 'https://raw.githubusercontent.com/Nakiyah24/ExplainableAI_Assignments/main/Assignment5/mango_leaf_dataset/disease_0002.JPG', 'https://raw.githubusercontent.com/Nakiyah24/ExplainableAI_Assignments/main/Assignment5/mango_leaf_dataset/disease_0003.JPG']
['https://raw.githubusercontent.com/Nakiyah24/ExplainableAI_Assignments/main/Assignment5/mango_leaf_dataset/healthy_0001.JPG', 'https://raw.githubusercontent.com/Nakiyah24/ExplainableAI_Assignments/main/Assignment5/mango_leaf_dataset/healthy_0002.JPG', 'https://raw.githubusercontent.com/Nakiyah24/ExplainableAI_Assignments/main/Assignment5/mango_leaf_dataset/healthy_0003.JPG']


Step 2:

Since I am calling the images from my github repo, the code below is just a simple sanity check to ensure I am able to load the images
(Code snippet take from Chatgpt-5 on 11/03/2025)

In [ ]:
def get_img(url):
    r = requests.get(url, timeout=15)
    r.raise_for_status()
    return Image.open(BytesIO(r.content)).convert("RGB")


# try first diseased + healthy
t1 = get_img(diseased_lst[0])
t2 = get_img(healthy_lst[0])
print("OK: both URLs load")

OK: both URLs load


Step 3: Preprocessing image

Now that the images are loaded, I need to preprocess and resize the image to 224×224 to match ResNet-50’s input. I will then convert the PIL image to a PyTorch tensor in [0,1], and normalize with the standard ImageNet mean/std so it matches what the pretrained model expects. I will also keep a float RGB copy for the Grad-CAM overlay, and finally add a batch dimension so the input is [1,3,224,224] for the forward pass.


Preprocessing code taken from Chatgpt-5 on 11/04/2025

In [ ]:
IM_SIZE = 224
tfm = transforms.Compose(
    [
        transforms.Resize((IM_SIZE, IM_SIZE)),
        transforms.ToTensor(),  # 2) PIL -> tensor in [0,1], shape [C,H,W]
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],  # 3) ImageNet normalization (per channel)
            std=[0.229, 0.224, 0.225],
        ),
    ]
)

In [ ]:
def prep(img):
    """
    img: PIL.Image
    returns:
      rgb -> numpy array, shape [224,224,3], values in [0,1], used only for overlay
      x   -> torch tensor, shape [1,3,224,224], normalized, ready for the model
    """
    rgb = np.array(img.resize((IM_SIZE, IM_SIZE)), dtype=np.float32) / 255.0
    x = tfm(img).unsqueeze(0)  # add batch dim -> [1,3,224,224]
    return rgb, x